#### Auteur: Irina Maslowski

### Projet N° 4 Segmentez des clients d'un site e-commerce

#### Données: [Brazilian E-Commerce Public Dataset by Olist. 100,000 Orders with product, customer and reviews info](https://www.kaggle.com/olistbr/brazilian-ecommerce)
années: 2016 - 2018

![Le schéma des données](scheme.png)

## Taches:
1. utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires
2. créer les meilleures features pour les exploiter à partir de données dans notre disposition
**Cahier des charges** :
1. La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
2. Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
3. Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
# EXEMPLE sur http://www.python-simple.com/python-statsmodels/statsmodels-anova.php
import statsmodels.formula.api
import statsmodels.api

In [2]:
# certains services de géolocalisation sont limités et payant au delà de certain nombre d'appel
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

## Sommaire:
* [Import des données](#first-bullet)
* [Etape 1 de merge](#merge1)
* [Etape 2 de merge](#merge2)
    * [Imputation de données de géolocalisation manquantes](#geo_fillin)
* [Etape 3 de merge](#merge3)
* [Etape 4 de merge](#merge4)
* [Etape 5 de merge](#merge5)
* [Etape 6 de merge](#merge6)
    * [Imputation d'une traduction manqaunte](#trad_prod)
* [Etape 7 de merge](#merge7)
* [Etape 8 de merge](#merge8)
* [Etape 9 de merge](#merge9)

# Import des données <a class="anchor" id="first-bullet"></a>

In [254]:
# olist_customers_dataset
df_olist_customers = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_customers_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
df_olist_customers.shape

(99441, 5)

In [5]:
df_olist_customers.duplicated(subset=['customer_id']).sum()

0

In [6]:
df_olist_customers.duplicated(subset=['customer_unique_id']).sum()

3345

In [7]:
df_olist_customers[df_olist_customers.duplicated(subset=['customer_unique_id'], keep=False) == True][['customer_unique_id', 'customer_id']].sort_values(by=['customer_unique_id'])

,customer_unique_id,customer_id
35608,00172711b30d52eea8b313a7f2cced02,24b0e2bd287e47d54d193e7bbb51103f
19299,00172711b30d52eea8b313a7f2cced02,1afe8a9c67eec3516c09a8bdcc539090
20023,004288347e5e88a27ded2bb23747066c,1b4a75b3478138e99902678254b260f4
22066,004288347e5e88a27ded2bb23747066c,f6efe5d5c7b85e12355f9d5c3db46da2
72451,004b45ec5c64187465168251cd1c9c2f,49cf243e0d353cd418ca77868e24a670
...,...,...
75057,ff922bdd6bafcdf99cb90d7f39cea5b3,1ae563fdfa500d150be6578066d83998
27992,ff922bdd6bafcdf99cb90d7f39cea5b3,bec0bf00ac5bee64ce8ef5283051a70c
79859,ff922bdd6bafcdf99cb90d7f39cea5b3,d064be88116eb8b958727aec4cf56a59
64323,ffe254cc039740e17dd15a5305035928,4b231c90751c27521f7ee27ed2dc3b8f


In [8]:
df_olist_customers.groupby('customer_id').agg({'customer_unique_id':'count'}).max()

customer_unique_id    1
dtype: int64

In [9]:
df_olist_customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [10]:
# olist_geolocation_dataset
df_olist_geolocation = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_geolocation_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [11]:
df_olist_geolocation[df_olist_geolocation['geolocation_zip_code_prefix'] == 14409]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
334422,14409,-20.509897,-47.397866,franca,SP
334449,14409,-20.497396,-47.399241,franca,SP
334459,14409,-20.510459,-47.399553,franca,SP
334462,14409,-20.480940,-47.394161,franca,SP
334463,14409,-20.515413,-47.398194,franca,SP
...,...,...,...,...,...
336625,14409,-20.483960,-47.403324,franca,SP
336650,14409,-20.483960,-47.403324,franca,SP
336653,14409,-20.487255,-47.405925,franca,SP
336657,14409,-20.510459,-47.399553,franca,SP


In [12]:
df_olist_geolocation[df_olist_geolocation['geolocation_zip_code_prefix'] == 14409]['geolocation_lat'].max()

-20.4688485

In [13]:
df_olist_geolocation[df_olist_geolocation['geolocation_zip_code_prefix'] == 14409]['geolocation_lat'].min()

-20.519565436705047

In [14]:
df_olist_geolocation[df_olist_geolocation['geolocation_zip_code_prefix'] == 14409]['geolocation_lat'].mean()

-20.498488755380297

In [15]:
df_olist_geolocation[df_olist_geolocation['geolocation_zip_code_prefix'] == 14409]['geolocation_lat'].median()

-20.502070116372256

les coordonnées sont suffisement proche pour qu'on puisse laisse une coordonnées médiane pour un zip code

In [16]:
df_olist_geolocation.shape

(1000163, 5)

#### suppression des doublons dans le fichiers des coordonnées

In [17]:
df_olist_geolocation.duplicated().sum()

261831

In [18]:
df_geo_without_duplicates = df_olist_geolocation.drop_duplicates()

In [19]:
df_geo_without_duplicates

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000155,99965,-28.180655,-52.034367,agua santa,RS
1000156,99950,-28.072188,-52.011272,tapejara,RS
1000157,99950,-28.068864,-52.012964,tapejara,RS
1000158,99950,-28.068639,-52.010705,tapejara,RS


In [20]:
df_geo_without_duplicates[df_geo_without_duplicates['geolocation_zip_code_prefix'] == 1046]['geolocation_lat'].median()

-23.54574166250217

In [21]:
df_geo_without_duplicates[df_geo_without_duplicates['geolocation_zip_code_prefix'] == 1046]['geolocation_lng'].median()

-46.643199191240036

In [22]:
df_geo_without_duplicates.duplicated(subset=['geolocation_zip_code_prefix']).sum()

719317

Aggrégation de cellules de zip-code en choisissant la médiane des coordonnées

In [23]:
df_geo_agg = df_geo_without_duplicates.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat':'median',
    'geolocation_lng':'median', 
    'geolocation_city': 'last',
    'geolocation_state': 'last'
}).reset_index()

In [24]:
df_geo_agg.duplicated(subset=['geolocation_zip_code_prefix']).sum()

0

In [25]:
df_geo_agg

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1001,-23.549951,-46.634027,são paulo,SP
1,1002,-23.548228,-46.635247,são paulo,SP
2,1003,-23.548977,-46.635313,sao paulo,SP
3,1004,-23.549550,-46.634771,sao paulo,SP
4,1005,-23.549763,-46.636100,sao paulo,SP
...,...,...,...,...,...
19010,99960,-27.953797,-52.029641,charrua,RS
19011,99965,-28.179542,-52.035551,agua santa,RS
19012,99970,-28.343257,-51.875470,ciriaco,RS
19013,99980,-28.388342,-51.846871,david canabarro,RS


In [26]:
# olist_order_items_dataset
df_olist_order_items = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_order_items_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [27]:
df_olist_order_items.shape

(112650, 7)

In [28]:
df_olist_order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [29]:
# olist_order_payments_dataset
df_olist_order_payments = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_order_payments_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [30]:
df_olist_order_payments.shape

(103886, 5)

In [225]:
df_olist_order_payments.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [31]:
# olist_order_reviews_dataset
df_olist_order_reviews_dataset = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_order_reviews_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_order_reviews_dataset.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [32]:
df_olist_order_reviews_dataset.shape

(100000, 7)

In [244]:
df_olist_order_reviews_dataset.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [245]:
df_olist_order_reviews_dataset['contain_message'] = df_olist_order_reviews_dataset['review_comment_message'].notna()

In [247]:
df_olist_order_reviews_dataset.drop(['review_comment_message'], axis=1, inplace=True)

In [248]:
df_olist_order_reviews_dataset.drop(['review_comment_title'], axis=1, inplace=True)

In [249]:
df_olist_order_reviews_dataset.isna().sum()

review_id                  0
order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
contain_message            0
dtype: int64

In [33]:
# olist_orders_dataset
df_olist_orders = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_orders_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [34]:
df_olist_orders.shape

(99441, 8)

In [35]:
# olist_products_dataset
df_olist_products = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_products_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [36]:
df_olist_products.shape

(32951, 9)

In [170]:
df_olist_products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [37]:
# olist_sellers_dataset
df_olist_sellers = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/olist_sellers_dataset.csv', encoding='utf-8', low_memory=False)
df_olist_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [38]:
df_olist_sellers.shape

(3095, 4)

In [113]:
df_olist_sellers.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [39]:
# product_category_name_translation
df_product_category_name_translation = pd.read_csv('D:/OpenClassroomsProjectData/e-commerce/product_category_name_translation.csv', encoding='utf-8', low_memory=False)
df_product_category_name_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [40]:
df_product_category_name_translation.shape

(71, 2)

In [171]:
df_product_category_name_translation.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

# Etape 1 de merge <a class="anchor" id="merge1"></a>

#### Merge de dataframe 'commandes' avec la dataframe 'clients'
dataframe *df_olist_orders_customers*

In [41]:
df_olist_orders_customers = df_olist_orders.merge(df_olist_customers,  how='left', on='customer_id')
df_olist_orders_customers.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP


#### Idées de feature engineering pour les commandes:
temps de livraison<br>
temps depuis la dernière commande

In [42]:
df_olist_orders_customers[df_olist_orders_customers['order_delivered_customer_date'].isna()][['order_delivered_customer_date','order_delivered_carrier_date', 'order_approved_at', 'order_status']]

,order_delivered_customer_date,order_delivered_carrier_date,order_approved_at,order_status
6,NaN,NaN,2017-04-13 13:25:17,invoiced
44,NaN,2018-06-05 14:32:00,2018-06-05 04:31:18,shipped
103,NaN,NaN,2018-08-07 06:15:14,invoiced
128,NaN,NaN,2017-09-03 14:30:09,processing
154,NaN,2018-01-11 19:39:23,2018-01-11 02:32:30,shipped
...,...,...,...,...
99283,NaN,NaN,NaN,canceled
99313,NaN,NaN,2018-01-09 07:18:05,processing
99347,NaN,NaN,NaN,canceled
99348,NaN,NaN,2017-08-28 15:44:47,unavailable


In [43]:
df_olist_orders_customers.shape

(99441, 12)

![Le schéma des données](scheme_etape_1.png)

# Etape 2 de merge <a class="anchor" id="merge2"></a>

#### Merge des dataframes df_olist_orders_customers et df_geo_agg
dataframe résultant ==> df_olist_orders_customers_geo

In [44]:
df_olist_orders_customers_geo = df_olist_orders_customers.merge(df_geo_agg, how='left', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')

In [45]:
df_ord_cust_geo = df_olist_orders_customers_geo.copy()

In [46]:
df_ord_cust_geo.shape

(99441, 17)

In [47]:
df_olist_orders_customers_geo.shape

(99441, 17)

In [48]:
df_olist_orders_customers_geo

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.575816,-46.587303,são paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813.0,-12.126533,-45.008244,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,75265.0,-16.744472,-48.514624,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,59296.0,-5.773971,-35.273838,são gonçalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,9195.0,-23.675471,-46.514788,santo andre,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,12209.0,-23.177048,-45.884506,sao jose dos campos,SP
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,11722.0,-24.001467,-46.449577,praia grande,SP
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,45920.0,-17.897354,-39.372100,nova vicosa,BA
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,28685.0,-22.562040,-42.690186,japuiba,RJ


In [49]:
# différences entre les villes
bd_diff_cities = df_olist_orders_customers_geo[df_olist_orders_customers_geo['customer_city'] != df_olist_orders_customers_geo['geolocation_city']][['customer_city', 'geolocation_city']]

In [50]:
bd_diff_cities

,customer_city,geolocation_city
0,sao paulo,são paulo
3,sao goncalo do amarante,são gonçalo do amarante
11,sao paulo,são paulo
20,sao paulo,são paulo
22,sao paulo,são paulo
...,...,...
99407,sao paulo,são paulo
99414,sao paulo,são paulo
99417,brasilia,cruzeiro
99426,brasilia,brasília


La différence entre les noms des villes apparait surtout suite à la forme normalisée dans *'customer_city'*

#### Nombre de geolocalisations manquantes

In [51]:
pd.isnull(df_olist_orders_customers_geo['geolocation_city']).sum()

278

In [52]:
# création de la df contenant que des lignes avec l'information manquante sur la géolocalisation
df_orders_cust_geona = df_olist_orders_customers_geo[df_olist_orders_customers_geo['geolocation_city'].isna()]

In [53]:
df_orders_cust_geona

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,NaN,NaN,NaN,NaN
610,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,NaN,NaN,NaN,NaN
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,NaN,NaN,NaN,NaN
926,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,NaN,NaN,NaN,NaN
1091,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97935,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,NaN,NaN,NaN,NaN
98886,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,NaN,NaN,NaN,NaN
99062,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,NaN,NaN,NaN,NaN
99162,e1d102f12364768054a07dfa10fc43cc,f58c14ad1417ae5e8f9c6d0f9f6aec24,delivered,2017-03-05 22:11:02,2017-03-05 22:25:16,2017-03-06 09:38:29,2017-03-23 15:06:37,2017-03-27 00:00:00,04a5a9b154c83e4b19258dcd0e4476ed,85118,palmeirinha,PR,NaN,NaN,NaN,NaN,NaN


In [54]:
# aggrégation des données de géolocalisation en fonction du nom de la ville
df_geofornan_agg = df_geo_without_duplicates.groupby('geolocation_city').agg({
    'geolocation_lat':'median',
    'geolocation_lng':'median', 
    'geolocation_state': 'last'
}).reset_index()

In [55]:
df_geofornan_agg

,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
0,* cidade,-25.571748,-49.334374,PR
1,...arraial do cabo,-22.969370,-42.029834,RJ
2,4o. centenario,-24.271860,-53.069433,PR
3,4º centenario,-24.279801,-53.074723,PR
4,abadia de goias,-16.766377,-49.435020,GO
...,...,...,...,...
8006,áurea,-27.695009,-52.055076,RS
8007,ângulo,-23.194603,-51.916767,PR
8008,érico cardoso,-13.393047,-42.134282,BA
8009,óbidos,-1.906778,-55.515603,PA


## Imputation de données manquantes de géolocalisation  <a class="anchor" id="geo_fillin"></a>

In [56]:
df_orders_cust_geona_pre_merge = df_orders_cust_geona.copy()

In [57]:
df_orders_cust_geona_pre_merge['old_index'] = df_orders_cust_geona_pre_merge.index

In [58]:
# remplissage des valeurs de géolocalisation manquantes en fonction de coordonnées associées aux noms des villes
df_orders_cust_geofilledna = df_orders_cust_geona_pre_merge.merge(df_geofornan_agg, how='left', left_on='customer_city', right_on='geolocation_city')

In [59]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,geolocation_zip_code_prefix,geolocation_lat_x,geolocation_lng_x,geolocation_city_x,geolocation_state_x,old_index,geolocation_city_y,geolocation_lat_y,geolocation_lng_y,geolocation_state_y
0,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,...,NaN,NaN,NaN,NaN,NaN,445,NaN,NaN,NaN,NaN
1,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,...,NaN,NaN,NaN,NaN,NaN,610,brasilia,-15.820008,-47.979953,DF
2,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,...,NaN,NaN,NaN,NaN,NaN,681,NaN,NaN,NaN,NaN
3,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,...,NaN,NaN,NaN,NaN,NaN,926,brasilia,-15.820008,-47.979953,DF
4,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,...,NaN,NaN,NaN,NaN,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,...,NaN,NaN,NaN,NaN,NaN,97935,sao mateus,-18.719300,-39.855049,ES
274,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,...,NaN,NaN,NaN,NaN,NaN,98886,brasilia,-15.820008,-47.979953,DF
275,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,...,NaN,NaN,NaN,NaN,NaN,99062,brasilia,-15.820008,-47.979953,DF
276,e1d102f12364768054a07dfa10fc43cc,f58c14ad1417ae5e8f9c6d0f9f6aec24,delivered,2017-03-05 22:11:02,2017-03-05 22:25:16,2017-03-06 09:38:29,2017-03-23 15:06:37,2017-03-27 00:00:00,04a5a9b154c83e4b19258dcd0e4476ed,85118,...,NaN,NaN,NaN,NaN,NaN,99162,NaN,NaN,NaN,NaN


In [60]:
df_orders_cust_geofilledna.drop(['geolocation_lat_x','geolocation_lng_x','geolocation_city_x','geolocation_state_x'], axis=1, inplace=True)

In [61]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city_y,geolocation_lat_y,geolocation_lng_y,geolocation_state_y
0,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,NaN,NaN,NaN
1,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,610,brasilia,-15.820008,-47.979953,DF
2,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,NaN,NaN,NaN
3,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,926,brasilia,-15.820008,-47.979953,DF
4,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,97935,sao mateus,-18.719300,-39.855049,ES
274,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,98886,brasilia,-15.820008,-47.979953,DF
275,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,99062,brasilia,-15.820008,-47.979953,DF
276,e1d102f12364768054a07dfa10fc43cc,f58c14ad1417ae5e8f9c6d0f9f6aec24,delivered,2017-03-05 22:11:02,2017-03-05 22:25:16,2017-03-06 09:38:29,2017-03-23 15:06:37,2017-03-27 00:00:00,04a5a9b154c83e4b19258dcd0e4476ed,85118,palmeirinha,PR,NaN,99162,NaN,NaN,NaN,NaN


In [62]:
df_orders_cust_geofilledna.rename(columns={"geolocation_city_y": "geolocation_city", "geolocation_lat_y": "geolocation_lat",
                                          "geolocation_lng_y": "geolocation_lng", "geolocation_state_y": "geolocation_state"},
                                  inplace = True)

In [63]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
0,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,NaN,NaN,NaN
1,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,610,brasilia,-15.820008,-47.979953,DF
2,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,NaN,NaN,NaN
3,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,926,brasilia,-15.820008,-47.979953,DF
4,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,97935,sao mateus,-18.719300,-39.855049,ES
274,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,98886,brasilia,-15.820008,-47.979953,DF
275,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,99062,brasilia,-15.820008,-47.979953,DF
276,e1d102f12364768054a07dfa10fc43cc,f58c14ad1417ae5e8f9c6d0f9f6aec24,delivered,2017-03-05 22:11:02,2017-03-05 22:25:16,2017-03-06 09:38:29,2017-03-23 15:06:37,2017-03-27 00:00:00,04a5a9b154c83e4b19258dcd0e4476ed,85118,palmeirinha,PR,NaN,99162,NaN,NaN,NaN,NaN


In [64]:
df_orders_cust_geofilledna.set_index(df_orders_cust_geofilledna['old_index'], inplace=True)

In [65]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
old_index,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,NaN,NaN,NaN
610,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,610,brasilia,-15.820008,-47.979953,DF
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,NaN,NaN,NaN
926,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,926,brasilia,-15.820008,-47.979953,DF
1091,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97935,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,97935,sao mateus,-18.719300,-39.855049,ES
98886,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,98886,brasilia,-15.820008,-47.979953,DF
99062,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,99062,brasilia,-15.820008,-47.979953,DF


In [66]:
df_orders_cust_geofilledna['geolocation_city'].isna().sum()

49

Il reste 49 NaN. On crée une nouvelle df ('df_orders_cust_geoetranger') avec ses lignes

In [67]:
df_orders_cust_geoetranger = df_orders_cust_geofilledna[df_orders_cust_geofilledna['geolocation_city'].isna()].copy()

In [68]:
list_cities = df_geo_without_duplicates['geolocation_city'].unique()

In [69]:
list_cities

array(['sao paulo', 'são paulo', 'sao bernardo do campo', ..., 'ciríaco',
       'estação', 'vila lângaro'], dtype=object)

In [70]:
with open('listfile.txt', 'w', encoding='utf8') as filehandle:
    for listitem in list_cities:
        filehandle.write('%s\n' % listitem)

In [71]:
# source de code: https://www.askpython.com/python/python-geopy-to-find-geocode-of-an-address
#Creating an instance of Nominatim Class
geolocator = Nominatim(user_agent="my_request")

In [72]:
#applying the rate limiter wrapper
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [73]:
df_orders_cust_geoetranger

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
old_index,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,NaN,NaN,NaN
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,NaN,NaN,NaN
5497,ae78d443f07cc7da4cb961f34f1bc6a8,aa4c5b95810a5d4b1bf2cc56e7d76cd6,delivered,2017-08-31 20:47:58,2017-09-01 22:33:01,2017-09-06 23:42:13,2017-10-02 11:04:04,2017-09-27 00:00:00,41db2509d2fd3e2fe68e88deedaa0c1b,59299,poco de pedra,RN,NaN,5497,NaN,NaN,NaN,NaN
6559,6c187a778a170f2bc6beac3979e3a81c,e37623a7c983c5174d5aea2aad30a080,delivered,2018-03-19 12:56:50,2018-03-21 02:51:11,2018-03-21 22:13:37,2018-04-16 18:14:28,2018-04-11 00:00:00,7d8edfd1d3efadef19da57a63d21ed92,83843,doce grande,PR,NaN,6559,NaN,NaN,NaN,NaN
7553,5050bad26c6099dce48e97368a53f149,f341549b5b28a46a5b4db2f4372e36a2,delivered,2017-05-13 15:39:44,2017-05-13 15:50:16,2017-05-19 10:10:28,2017-05-23 15:38:02,2017-05-24 00:00:00,b085fe730254d7b519bc1231da539c38,6930,cipo-guacu,SP,NaN,7553,NaN,NaN,NaN,NaN
10699,7bac1567c0592387d6b83a189f7d25ef,4a5642b29f7d0885758928dc7ec35909,delivered,2018-02-14 17:25:43,2018-02-14 17:35:32,2018-02-21 20:27:15,2018-03-14 18:51:44,2018-03-16 00:00:00,04f08b09dd9ae5d658be400b1b786c47,75784,domiciano ribeiro,GO,NaN,10699,NaN,NaN,NaN,NaN
14006,7a36ddcacbdd80a7dd8771aa4c6e465e,e988b94fc82408ecac8299c744959c58,delivered,2018-08-06 06:42:46,2018-08-06 06:50:13,2018-08-07 07:38:00,2018-08-16 19:25:28,2018-08-27 00:00:00,d01e20bfb741f8a5a4c2c8582617c580,29718,angelo frechiani,ES,NaN,14006,NaN,NaN,NaN,NaN
14664,2ee460773e708be4e0208745a3864b8c,3bd12f7c1ad3a3908905785e43de8603,shipped,2017-06-22 22:34:26,2017-06-22 22:50:09,2017-06-28 15:17:48,NaN,2017-08-07 00:00:00,2e5fb502adb87cf265468f4d2096d724,65137,maioba,MA,NaN,14664,NaN,NaN,NaN,NaN
17005,971e55ce6ef3eae4af8292bd71c8e27f,8a629de914739aa508711c4311abf537,delivered,2018-03-28 12:08:31,2018-03-28 12:15:23,2018-04-02 16:16:52,2018-04-18 19:48:27,2018-04-19 00:00:00,33af0bbf265311b13bf09d859309b52b,28575,jaguarembe,RJ,NaN,17005,NaN,NaN,NaN,NaN


In [74]:
# test de cancatenation de l'information pour plus de précision dans le résultat
#df_orders_cust_geoetranger['full_adress'] = df_orders_cust_geoetranger['customer_zip_code_prefix'].map(str) + ', ' + df_orders_cust_geoetranger['customer_city']

In [75]:
#Applying the method to pandas DataFrame
df_orders_cust_geoetranger['location'] = df_orders_cust_geoetranger['customer_city'].apply(geocode)
df_orders_cust_geoetranger['geolocation_lat'] = df_orders_cust_geoetranger['location'].apply(lambda x: x.latitude if x else None)
df_orders_cust_geoetranger['geolocation_lng'] = df_orders_cust_geoetranger['location'].apply(lambda x: x.longitude if x else None)

In [76]:
df_orders_cust_geoetranger

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state,location
old_index,,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,-16.715183,-43.854449,NaN,"(Alto São João, Montes Claros, Microrregião Mo..."
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,-21.214826,-41.467473,NaN,"(Santo Eduardo, Campos dos Goytacazes, Região ..."
5497,ae78d443f07cc7da4cb961f34f1bc6a8,aa4c5b95810a5d4b1bf2cc56e7d76cd6,delivered,2017-08-31 20:47:58,2017-09-01 22:33:01,2017-09-06 23:42:13,2017-10-02 11:04:04,2017-09-27 00:00:00,41db2509d2fd3e2fe68e88deedaa0c1b,59299,poco de pedra,RN,NaN,5497,NaN,-9.045909,-41.033095,NaN,"(Poço de Pedra, Casa Nova, Microrregião de Jua..."
6559,6c187a778a170f2bc6beac3979e3a81c,e37623a7c983c5174d5aea2aad30a080,delivered,2018-03-19 12:56:50,2018-03-21 02:51:11,2018-03-21 22:13:37,2018-04-16 18:14:28,2018-04-11 00:00:00,7d8edfd1d3efadef19da57a63d21ed92,83843,doce grande,PR,NaN,6559,NaN,-26.022387,-49.430872,NaN,"(Doce Grande, Quitandinha, Região Geográfica I..."
7553,5050bad26c6099dce48e97368a53f149,f341549b5b28a46a5b4db2f4372e36a2,delivered,2017-05-13 15:39:44,2017-05-13 15:50:16,2017-05-19 10:10:28,2017-05-23 15:38:02,2017-05-24 00:00:00,b085fe730254d7b519bc1231da539c38,6930,cipo-guacu,SP,NaN,7553,NaN,-23.857032,-46.792861,NaN,"(Cipó-Guaçu, Embu-Guaçu, Região Imediata de Sã..."
10699,7bac1567c0592387d6b83a189f7d25ef,4a5642b29f7d0885758928dc7ec35909,delivered,2018-02-14 17:25:43,2018-02-14 17:35:32,2018-02-21 20:27:15,2018-03-14 18:51:44,2018-03-16 00:00:00,04f08b09dd9ae5d658be400b1b786c47,75784,domiciano ribeiro,GO,NaN,10699,NaN,-16.920938,-47.707508,NaN,"(Domiciano Ribeiro, Ipameri, Região Geográfica..."
14006,7a36ddcacbdd80a7dd8771aa4c6e465e,e988b94fc82408ecac8299c744959c58,delivered,2018-08-06 06:42:46,2018-08-06 06:50:13,2018-08-07 07:38:00,2018-08-16 19:25:28,2018-08-27 00:00:00,d01e20bfb741f8a5a4c2c8582617c580,29718,angelo frechiani,ES,NaN,14006,NaN,-19.304460,-40.668167,NaN,"(Ângelo Frechiani, Colatina, Região Geográfica..."
14664,2ee460773e708be4e0208745a3864b8c,3bd12f7c1ad3a3908905785e43de8603,shipped,2017-06-22 22:34:26,2017-06-22 22:50:09,2017-06-28 15:17:48,NaN,2017-08-07 00:00:00,2e5fb502adb87cf265468f4d2096d724,65137,maioba,MA,NaN,14664,NaN,-2.523879,-44.178247,NaN,"(Maioba, Paço do Lumiar, Região Geográfica Ime..."
17005,971e55ce6ef3eae4af8292bd71c8e27f,8a629de914739aa508711c4311abf537,delivered,2018-03-28 12:08:31,2018-03-28 12:15:23,2018-04-02 16:16:52,2018-04-18 19:48:27,2018-04-19 00:00:00,33af0bbf265311b13bf09d859309b52b,28575,jaguarembe,RJ,NaN,17005,NaN,-21.724840,-41.992431,NaN,"(Jaguarembé, Itaocara, Região Geográfica Imedi..."


**Observation** : à priori, on remarque qu'une seule erreur à l'index 59563, car les coordonnées trouvées sont en France

Car c'est qu'une seule ligne, je la remplie à la main, en récupérant les coordonnées à https://www.coordonnees-gps.fr/

In [77]:
df_orders_cust_geoetranger.loc[59563,'geolocation_lat'] = -3.622443

In [78]:
df_orders_cust_geoetranger.loc[59563,'geolocation_lng'] = -39.821936

In [79]:
df_orders_cust_geoetranger.drop(['location'], axis=1, inplace=True)

In [80]:
df_orders_cust_geoetranger

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
old_index,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,-16.715183,-43.854449,NaN
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,-21.214826,-41.467473,NaN
5497,ae78d443f07cc7da4cb961f34f1bc6a8,aa4c5b95810a5d4b1bf2cc56e7d76cd6,delivered,2017-08-31 20:47:58,2017-09-01 22:33:01,2017-09-06 23:42:13,2017-10-02 11:04:04,2017-09-27 00:00:00,41db2509d2fd3e2fe68e88deedaa0c1b,59299,poco de pedra,RN,NaN,5497,NaN,-9.045909,-41.033095,NaN
6559,6c187a778a170f2bc6beac3979e3a81c,e37623a7c983c5174d5aea2aad30a080,delivered,2018-03-19 12:56:50,2018-03-21 02:51:11,2018-03-21 22:13:37,2018-04-16 18:14:28,2018-04-11 00:00:00,7d8edfd1d3efadef19da57a63d21ed92,83843,doce grande,PR,NaN,6559,NaN,-26.022387,-49.430872,NaN
7553,5050bad26c6099dce48e97368a53f149,f341549b5b28a46a5b4db2f4372e36a2,delivered,2017-05-13 15:39:44,2017-05-13 15:50:16,2017-05-19 10:10:28,2017-05-23 15:38:02,2017-05-24 00:00:00,b085fe730254d7b519bc1231da539c38,6930,cipo-guacu,SP,NaN,7553,NaN,-23.857032,-46.792861,NaN
10699,7bac1567c0592387d6b83a189f7d25ef,4a5642b29f7d0885758928dc7ec35909,delivered,2018-02-14 17:25:43,2018-02-14 17:35:32,2018-02-21 20:27:15,2018-03-14 18:51:44,2018-03-16 00:00:00,04f08b09dd9ae5d658be400b1b786c47,75784,domiciano ribeiro,GO,NaN,10699,NaN,-16.920938,-47.707508,NaN
14006,7a36ddcacbdd80a7dd8771aa4c6e465e,e988b94fc82408ecac8299c744959c58,delivered,2018-08-06 06:42:46,2018-08-06 06:50:13,2018-08-07 07:38:00,2018-08-16 19:25:28,2018-08-27 00:00:00,d01e20bfb741f8a5a4c2c8582617c580,29718,angelo frechiani,ES,NaN,14006,NaN,-19.304460,-40.668167,NaN
14664,2ee460773e708be4e0208745a3864b8c,3bd12f7c1ad3a3908905785e43de8603,shipped,2017-06-22 22:34:26,2017-06-22 22:50:09,2017-06-28 15:17:48,NaN,2017-08-07 00:00:00,2e5fb502adb87cf265468f4d2096d724,65137,maioba,MA,NaN,14664,NaN,-2.523879,-44.178247,NaN
17005,971e55ce6ef3eae4af8292bd71c8e27f,8a629de914739aa508711c4311abf537,delivered,2018-03-28 12:08:31,2018-03-28 12:15:23,2018-04-02 16:16:52,2018-04-18 19:48:27,2018-04-19 00:00:00,33af0bbf265311b13bf09d859309b52b,28575,jaguarembe,RJ,NaN,17005,NaN,-21.724840,-41.992431,NaN


In [81]:
#  liste de l'index des lignes où la latitude contient NaN
index_geonull = df_orders_cust_geofilledna[df_orders_cust_geofilledna['geolocation_lat'].isna()].index

In [82]:
df_orders_cust_geofilledna.loc[index_geonull, 'geolocation_lat'] = df_orders_cust_geoetranger['geolocation_lat']

In [83]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
old_index,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,-16.715183,NaN,NaN
610,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,610,brasilia,-15.820008,-47.979953,DF
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,-21.214826,NaN,NaN
926,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,926,brasilia,-15.820008,-47.979953,DF
1091,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97935,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,97935,sao mateus,-18.719300,-39.855049,ES
98886,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,98886,brasilia,-15.820008,-47.979953,DF
99062,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,99062,brasilia,-15.820008,-47.979953,DF


In [84]:
df_orders_cust_geofilledna['geolocation_lat'].isna().sum()

0

In [85]:
df_orders_cust_geofilledna.loc[index_geonull, 'geolocation_lng'] = df_orders_cust_geoetranger['geolocation_lng']

In [86]:
df_orders_cust_geofilledna['geolocation_lng'].isna().sum()

0

In [87]:
df_orders_cust_geofilledna

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,old_index,geolocation_city,geolocation_lat,geolocation_lng,geolocation_state
old_index,,,,,,,,,,,,,,,,,,
445,690199d6a2c51ff57c6b392d7680cbfd,19bacb562bd43bd4eaf05b6c0a59dad0,delivered,2018-03-16 11:31:18,2018-03-16 11:49:14,2018-03-19 19:56:23,2018-05-14 12:28:56,2018-04-11 00:00:00,c5f01991eadc43c924dfd891e9026217,87323,alto sao joao,PR,NaN,445,NaN,-16.715183,-43.854449,NaN
610,9f2da077d2d1eb6e8abcacef8776e941,2386792ff4818fe030a3b1364994f51e,delivered,2017-10-28 21:26:26,2017-10-28 21:46:29,2017-11-06 11:08:52,2017-11-13 22:33:12,2017-11-22 00:00:00,ab7bed4092cc3dadbaefba86cb45f37f,73081,brasilia,DF,NaN,610,brasilia,-15.820008,-47.979953,DF
681,1b00c4d80c81cdbc36487c96c6e92f07,6f392cfb40b84e0857b16c23c773aa31,delivered,2017-05-14 09:36:36,2017-05-16 03:22:50,2017-05-16 12:05:28,2017-05-23 07:52:33,2017-06-02 00:00:00,5399c9213ca365a480537a1b21423d2c,28160,santo eduardo,RJ,NaN,681,NaN,-21.214826,-41.467473,NaN
926,7ebe19943d85e81a7b0eb108681276b7,fa3a8736d5ae8f7df9b0a4154d0b14dc,delivered,2018-07-22 10:28:14,2018-07-23 11:31:43,2018-07-23 10:09:00,2018-07-26 16:42:42,2018-08-15 00:00:00,2de521d3fd097acfbd5bed9c7b9ed0d0,71551,brasilia,DF,NaN,926,brasilia,-15.820008,-47.979953,DF
1091,d0bec96aad189992b278688279ba1511,f4302056f0c58570522590f8181de2c7,delivered,2018-05-08 21:04:16,2018-05-08 21:15:28,2018-05-10 18:29:00,2018-05-21 13:06:35,2018-06-05 00:00:00,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,1091,picos,-7.083213,-41.468344,PI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97935,75f6ded14ff770cff5dab61532421db4,bbf5e98dabf1bdac752afc68f8032bda,delivered,2018-04-03 08:37:21,2018-04-03 08:50:14,2018-04-04 19:28:19,2018-05-07 22:54:47,2018-04-20 00:00:00,13aba123512eef450a11646c572fccca,29949,sao mateus,ES,NaN,97935,sao mateus,-18.719300,-39.855049,ES
98886,da47f3e1d422ef0acfa45d3a81f0085f,d14b76c93d237241c52e0e6767feb42a,delivered,2017-12-04 21:53:42,2017-12-05 10:30:46,2017-12-06 16:58:51,2017-12-19 16:23:35,2018-01-02 00:00:00,4a88a750280bdba0dd1f167ae2b68bfd,71919,brasilia,DF,NaN,98886,brasilia,-15.820008,-47.979953,DF
99062,1a0e54c67a7d784f932f5cc4f953fbaf,baca33004aa726524d5a891853100559,delivered,2017-06-16 21:56:39,2017-06-16 22:06:26,2017-06-20 15:30:45,2017-07-03 08:49:44,2017-07-13 00:00:00,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF,NaN,99062,brasilia,-15.820008,-47.979953,DF


La dataframe *df_orders_cust_geofilledna* ne contient plus de NaN dans les colonnes *geolocation_lat* et *geolocation_lng*

In [88]:
df_orders_cust_geofilledna.shape

(278, 18)

In [89]:
df_olist_orders_customers_geo['geolocation_lat'].isna().sum()

278

In [90]:
df_olist_orders_customers_geo['geolocation_lng'].isna().sum()

278

In [91]:
#  liste de l'index des lignes où la latitude contient NaN dans le dataframe 'df_olist_orders_customers_geo'
index_df_olist_geonull = df_olist_orders_customers_geo[df_olist_orders_customers_geo['geolocation_lat'].isna()].index

In [92]:
index_df_olist_geonull

Int64Index([  445,   610,   681,   926,  1091,  1202,  1343,  2004,  2344,
             2495,
            ...
            92991, 94493, 95336, 95628, 95962, 97935, 98886, 99062, 99162,
            99217],
           dtype='int64', length=278)

In [93]:
df_orders_cust_geofilledna.index

Int64Index([  445,   610,   681,   926,  1091,  1202,  1343,  2004,  2344,
             2495,
            ...
            92991, 94493, 95336, 95628, 95962, 97935, 98886, 99062, 99162,
            99217],
           dtype='int64', name='old_index', length=278)

In [94]:
df_ord_cust_geo.loc[index_df_olist_geonull, 'geolocation_lat'] = df_orders_cust_geofilledna['geolocation_lat']

In [95]:
df_ord_cust_geo.loc[index_df_olist_geonull, 'geolocation_lng'] = df_orders_cust_geofilledna['geolocation_lng']

In [96]:
df_ord_cust_geo['geolocation_lat'].isna().sum()

0

In [97]:
df_ord_cust_geo['geolocation_lng'].isna().sum()

0

In [98]:
df_ord_cust_geo.shape

(99441, 17)

In [99]:
df_ord_cust_geo

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.575816,-46.587303,são paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813.0,-12.126533,-45.008244,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,75265.0,-16.744472,-48.514624,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,59296.0,-5.773971,-35.273838,são gonçalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,9195.0,-23.675471,-46.514788,santo andre,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,12209.0,-23.177048,-45.884506,sao jose dos campos,SP
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,11722.0,-24.001467,-46.449577,praia grande,SP
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,45920.0,-17.897354,-39.372100,nova vicosa,BA
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,28685.0,-22.562040,-42.690186,japuiba,RJ


In [100]:
df_ord_cust_geo.drop(['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'], axis=1, inplace=True)

In [101]:
df_ord_cust_geo.rename(columns={"geolocation_lat": "customer_lat", "geolocation_lng": "customer_lng"}, inplace = True)

In [102]:
df_ord_cust_geo

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.575816,-46.587303
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.126533,-45.008244
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.744472,-48.514624
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,-5.773971,-35.273838
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,-23.675471,-46.514788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,-23.177048,-45.884506
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,-24.001467,-46.449577
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,-17.897354,-39.372100
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,-22.562040,-42.690186


![Le schéma des données](scheme_etape_2.png)

# Etape 3 de merge <a class="anchor" id="merge3"></a>

### Merge de *df_ord_cust_geo* avec *df_olist_order_items*

In [103]:
df_ord_cust_geo

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.575816,-46.587303
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.126533,-45.008244
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.744472,-48.514624
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,-5.773971,-35.273838
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,-23.675471,-46.514788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,-23.177048,-45.884506
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,-24.001467,-46.449577
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,-17.897354,-39.372100
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,-22.562040,-42.690186


In [104]:
# df_olist_orders_customers.merge(df_geo_agg, how='left', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
df_orders_cust_geo_orderitems = df_ord_cust_geo.merge(df_olist_order_items, how='left', on = 'order_id')

In [105]:
df_orders_cust_geo_orderitems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113425 entries, 0 to 113424
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113425 non-null  object 
 1   customer_id                    113425 non-null  object 
 2   order_status                   113425 non-null  object 
 3   order_purchase_timestamp       113425 non-null  object 
 4   order_approved_at              113264 non-null  object 
 5   order_delivered_carrier_date   111457 non-null  object 
 6   order_delivered_customer_date  110196 non-null  object 
 7   order_estimated_delivery_date  113425 non-null  object 
 8   customer_unique_id             113425 non-null  object 
 9   customer_zip_code_prefix       113425 non-null  int64  
 10  customer_city                  113425 non-null  object 
 11  customer_state                 113425 non-null  object 
 12  customer_lat                  

In [106]:
df_orders_cust_geo_orderitems

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.575816,-46.587303,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.126533,-45.008244,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.744472,-48.514624,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,-5.773971,-35.273838,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,-23.675471,-46.514788,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113420,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,-24.001467,-46.449577,1.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
113421,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,-17.897354,-39.372100,1.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
113422,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,-22.562040,-42.690186,1.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
113423,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,-22.562040,-42.690186,2.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59


In [107]:
df_orders_cust_geo_orderitems.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
customer_lat                        0
customer_lng                        0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
dtype: int64

In [108]:
df_orders_cust_geo_orderitems[df_orders_cust_geo_orderitems['order_item_id'].isna()]['order_status'].unique()

array(['unavailable', 'canceled', 'created', 'shipped', 'invoiced'],
      dtype=object)

In [109]:
df_orders_cust_geo_orderitems.groupby('customer_id').agg({'order_id':'count'}).max()

order_id    21
dtype: int64

In [110]:
df_orders_cust_geo_orderitems.groupby('customer_unique_id').agg({'order_id':'count'}).max()

order_id    24
dtype: int64

In [111]:
df_orders_cust_geo_orderitems.groupby('customer_unique_id').agg({'customer_id':'count'}).max()

customer_id    24
dtype: int64

In [112]:
df_orders_cust_geo_orderitems.groupby('customer_id').agg({'customer_unique_id':'count'}).max()

customer_unique_id    21
dtype: int64

![Le schéma des données](scheme_etape_3.png)

# Etape 4 de merge <a class="anchor" id="merge4"></a>

### Merge de olist_sellers_dataset et olist_geolocation_dataset

In [114]:
df_olist_sellers

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [115]:
df_olist_sellers_geo_agg = df_olist_sellers.merge(df_geo_agg, how='left', left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')

In [117]:
# nombre de vendeurs avec l'information sur la géolocalisation manquante
df_olist_sellers_geo_agg['geolocation_lat'].isna().sum()

7

In [141]:
# index des lignes geolocation_lat nulles
index_sellers_geona = df_olist_sellers_geo_agg[df_olist_sellers_geo_agg['geolocation_lat'].isna()].index

In [142]:
index_sellers_geona

Int64Index([473, 791, 1672, 1931, 2182, 2986, 3028], dtype='int64')

In [120]:
df_sellers_geona = df_olist_sellers_geo_agg[df_olist_sellers_geo_agg['geolocation_lat'].isna()]

In [123]:
df_sellers_geona_pre_merge = df_sellers_geona.copy()

In [124]:
df_sellers_geona_pre_merge['old_index'] = df_sellers_geona_pre_merge.index

In [143]:
# remplissage des valeurs de géolocalisation manquantes en fonction de coordonnées associées aux noms des villes
df_sellers_geofilledna = df_sellers_geona_pre_merge.merge(df_geofornan_agg, how='left', left_on='seller_city', right_on='geolocation_city')

In [145]:
df_sellers_geofilledna.drop(['geolocation_lat_x','geolocation_lng_x','geolocation_city_x','geolocation_state_x', 'geolocation_city_y', 'geolocation_state_y'], axis=1, inplace=True)

In [146]:
df_sellers_geofilledna

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,old_index,geolocation_lat_y,geolocation_lng_y
0,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR,NaN,473,-25.449140,-49.271910
1,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS,NaN,791,-30.037884,-51.193383
2,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF,NaN,1672,-15.820008,-47.979953
3,2e90cb1677d35cfe24eef47d441b7c87,2285,sao paulo,SP,NaN,1931,-23.562440,-46.645713
4,0b3f27369a4d8df98f7eb91077e438ac,7412,aruja,SP,NaN,2182,-23.400984,-46.321737
5,42bde9fef835393bb8a8849cb6b7f245,71551,brasilia,DF,NaN,2986,-15.820008,-47.979953
6,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG,NaN,3028,-21.792327,-46.564103


In [147]:
df_sellers_geofilledna.rename(columns={"geolocation_lat_y": "geolocation_lat",
                                          "geolocation_lng_y": "geolocation_lng"}, inplace = True)

In [149]:
df_sellers_geofilledna.set_index(df_sellers_geofilledna['old_index'], inplace=True)

In [150]:
df_sellers_geofilledna

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,old_index,geolocation_lat,geolocation_lng
old_index,,,,,,,,
473,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR,NaN,473,-25.449140,-49.271910
791,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS,NaN,791,-30.037884,-51.193383
1672,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF,NaN,1672,-15.820008,-47.979953
1931,2e90cb1677d35cfe24eef47d441b7c87,2285,sao paulo,SP,NaN,1931,-23.562440,-46.645713
2182,0b3f27369a4d8df98f7eb91077e438ac,7412,aruja,SP,NaN,2182,-23.400984,-46.321737
2986,42bde9fef835393bb8a8849cb6b7f245,71551,brasilia,DF,NaN,2986,-15.820008,-47.979953
3028,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG,NaN,3028,-21.792327,-46.564103


In [152]:
df_olist_sellers_geo_agg.loc[index_sellers_geona, 'geolocation_lat'] = df_sellers_geofilledna['geolocation_lat']

In [153]:
df_olist_sellers_geo_agg[df_olist_sellers_geo_agg['geolocation_lng'].isna()]

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
473,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR,NaN,-25.449140,NaN,NaN,NaN
791,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS,NaN,-30.037884,NaN,NaN,NaN
1672,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF,NaN,-15.820008,NaN,NaN,NaN
1931,2e90cb1677d35cfe24eef47d441b7c87,2285,sao paulo,SP,NaN,-23.562440,NaN,NaN,NaN
2182,0b3f27369a4d8df98f7eb91077e438ac,7412,aruja,SP,NaN,-23.400984,NaN,NaN,NaN
2986,42bde9fef835393bb8a8849cb6b7f245,71551,brasilia,DF,NaN,-15.820008,NaN,NaN,NaN
3028,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG,NaN,-21.792327,NaN,NaN,NaN


In [154]:
df_olist_sellers_geo_agg.loc[index_sellers_geona, 'geolocation_lng'] = df_sellers_geofilledna['geolocation_lng']

In [155]:
df_olist_sellers_geo_agg.isna().sum()

seller_id                      0
seller_zip_code_prefix         0
seller_city                    0
seller_state                   0
geolocation_zip_code_prefix    7
geolocation_lat                0
geolocation_lng                0
geolocation_city               7
geolocation_state              7
dtype: int64

In [156]:
df_olist_sellers_geo_agg.drop(['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'], axis=1, inplace=True)

In [157]:
df_olist_sellers_geo_agg.rename(columns={"geolocation_lat": "seller_lat",
                                          "geolocation_lng": "seller_lng"}, inplace = True)

In [158]:
df_olist_sellers_geo = df_olist_sellers_geo_agg.copy()

In [160]:
df_olist_sellers_geo.duplicated().sum()

0

In [162]:
df_olist_sellers_geo.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
seller_lat                0
seller_lng                0
dtype: int64

In [165]:
df_olist_sellers_geo

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_lat,seller_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.893863,-47.062006
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.382869,-46.947992
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.910174,-43.176775
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,-23.656991,-46.612490
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,-22.963763,-46.534676
...,...,...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,-23.448591,-51.869229
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC,-27.626905,-48.674389
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,-23.657622,-46.677126
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,-31.751451,-52.323078


![Le schéma des données](scheme_etape_4.png)

# Etape 5 de merge <a class="anchor" id="merge5"></a>

### merge de *df_orders_cust_geo_orderitems* et *df_olist_sellers_geo*

In [167]:
# seller_id
df_orders_cust_orderitems_sellers_geo = df_orders_cust_geo_orderitems.merge(df_olist_sellers_geo, how='left', on='seller_id')

In [168]:
df_orders_cust_orderitems_sellers_geo.shape

(113425, 25)

In [169]:
df_orders_cust_orderitems_sellers_geo.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
customer_lat                        0
customer_lng                        0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
seller_zip_code_prefix            775
seller_city                       775
seller_state                      775
seller_lat                        775
seller_lng                        775
dtype: int64

![Le schéma des données](scheme_etape_5.png)

# Etape 6 de merge <a class="anchor" id="merge6"></a>

### merge de *df_olist_products* avec *df_product_category_name_translation*

In [186]:
df_olist_products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [181]:
df_olist_products['product_category_name'].nunique()

73

In [174]:
df_product_category_name_translation.shape

(71, 2)

In [185]:
df_product_category_name_translation.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [183]:
df_olist_products_product_category_name_translation = df_olist_products.merge(df_product_category_name_translation, how='left', on='product_category_name')

In [189]:
df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'].notna() & df_olist_products_product_category_name_translation['product_category_name_english'].isna()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
1628,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59.0,621.0,4.0,2839.0,19.0,16.0,18.0,NaN
5821,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos,52.0,280.0,1.0,1200.0,25.0,33.0,25.0,NaN
7325,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos,58.0,284.0,1.0,1200.0,25.0,33.0,25.0,NaN
7478,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60.0,1532.0,3.0,650.0,16.0,22.0,20.0,NaN
8819,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos,54.0,382.0,2.0,850.0,30.0,21.0,22.0,NaN
11039,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos,46.0,280.0,1.0,1200.0,25.0,33.0,25.0,NaN
14266,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos,59.0,927.0,1.0,10600.0,40.0,20.0,38.0,NaN
16182,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos,58.0,792.0,4.0,750.0,30.0,30.0,30.0,NaN
16930,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60.0,840.0,6.0,800.0,18.0,22.0,22.0,NaN
17800,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos,52.0,1372.0,5.0,7350.0,40.0,30.0,23.0,NaN


## Imputation d'une traduction manqaunte <a class="anchor" id="trad_prod"></a>

In [198]:
# portateis_cozinha_e_preparadores_de_alimentos = portable_kitchen_and_food_preparers
df_olist_products_product_category_name_translation.loc[df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos'].index, 'product_category_name_english'] = 'portable_kitchen_and_food_preparers' 

In [199]:
df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
5821,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos,52.0,280.0,1.0,1200.0,25.0,33.0,25.0,portable_kitchen_and_food_preparers
7325,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos,58.0,284.0,1.0,1200.0,25.0,33.0,25.0,portable_kitchen_and_food_preparers
8819,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos,54.0,382.0,2.0,850.0,30.0,21.0,22.0,portable_kitchen_and_food_preparers
11039,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos,46.0,280.0,1.0,1200.0,25.0,33.0,25.0,portable_kitchen_and_food_preparers
14266,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos,59.0,927.0,1.0,10600.0,40.0,20.0,38.0,portable_kitchen_and_food_preparers
16182,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos,58.0,792.0,4.0,750.0,30.0,30.0,30.0,portable_kitchen_and_food_preparers
17800,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos,52.0,1372.0,5.0,7350.0,40.0,30.0,23.0,portable_kitchen_and_food_preparers
18610,7afdd65f79f63819ff5bee328843fa37,portateis_cozinha_e_preparadores_de_alimentos,48.0,305.0,1.0,750.0,20.0,20.0,20.0,portable_kitchen_and_food_preparers
26890,a4756663d007b0cd1af865754d08d968,portateis_cozinha_e_preparadores_de_alimentos,60.0,1304.0,4.0,650.0,22.0,6.0,14.0,portable_kitchen_and_food_preparers
29919,cb9d764f38ee4d0c00af64d5c388f837,portateis_cozinha_e_preparadores_de_alimentos,57.0,306.0,1.0,625.0,28.0,27.0,24.0,portable_kitchen_and_food_preparers


In [202]:
df_olist_products_product_category_name_translation.loc[df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'] == 'pc_gamer'].index, 'product_category_name_english'] = 'pc_gamer'

In [203]:
df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'] == 'pc_gamer']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
1628,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59.0,621.0,4.0,2839.0,19.0,16.0,18.0,pc_gamer
7478,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60.0,1532.0,3.0,650.0,16.0,22.0,20.0,pc_gamer
16930,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60.0,840.0,6.0,800.0,18.0,22.0,22.0,pc_gamer


In [205]:
df_olist_products_product_category_name_translation.drop(['product_category_name'], axis=1, inplace=True)

In [207]:
df_olist_products_product_category_name_translation.rename(columns={"product_category_name_english": "product_category_name"}, inplace=True)

In [219]:
df_olist_products_product_category_name_translation.loc[df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'].isna()].index, 'product_category_name'] = 'other'

In [220]:
df_olist_products_product_category_name_translation[df_olist_products_product_category_name_translation['product_category_name'] == 'other']

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,650.0,17.0,14.0,12.0,other
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,300.0,16.0,7.0,20.0,other
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,200.0,20.0,20.0,20.0,other
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,18500.0,41.0,30.0,41.0,other
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,300.0,35.0,7.0,12.0,other
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,NaN,NaN,NaN,1800.0,30.0,20.0,70.0,other
32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,NaN,NaN,800.0,30.0,10.0,23.0,other
32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,NaN,NaN,200.0,21.0,8.0,16.0,other
32772,fa51e914046aab32764c41356b9d4ea4,NaN,NaN,NaN,1300.0,45.0,16.0,45.0,other


In [221]:
df_olist_products_product_category_name_translation.isna().sum()

product_id                      0
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
product_category_name           0
dtype: int64

In [222]:
df_olist_products_category_name_en = df_olist_products_product_category_name_translation.copy()

# Etape 7 de merge <a class="anchor" id="merge7"></a>

### merge de *df_orders_cust_orderitems_sellers_geo* avec *df_olist_products_category_name_en*

In [223]:
df_orders_cust_orderitems_sellers_geo_products_en = df_orders_cust_orderitems_sellers_geo.merge(df_olist_products_category_name_en, how='left', on='product_id')

In [242]:
len(df_orders_cust_orderitems_sellers_geo_products_en)

113425

In [224]:
df_orders_cust_orderitems_sellers_geo_products_en.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
customer_lat                        0
customer_lng                        0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
seller_zip_code_prefix            775
seller_city                       775
seller_state                      775
seller_lat                        775
seller_lng                        775
product_name_lenght              2378
product_desc

![Le schéma des données](scheme_etape_7.png)

# Etape 8 de merge <a class="anchor" id="merge8"></a>

###  merge de *df_orders_cust_orderitems_sellers_geo_products_en* et *df_olist_order_payments*

In [226]:
df_orders_cust_orderitems_sellers_geo_products_en_order_payments = df_orders_cust_orderitems_sellers_geo_products_en.merge(df_olist_order_payments, how='left', on='order_id')

In [227]:
df_orders_cust_orderitems_sellers_geo_products_en_order_payments.shape

(118434, 37)

In [228]:
df_orders_cust_orderitems_sellers_geo_products_en_order_payments.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 176
order_delivered_carrier_date     2074
order_delivered_customer_date    3397
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
customer_lat                        0
customer_lng                        0
order_item_id                     830
product_id                        830
seller_id                         830
shipping_limit_date               830
price                             830
freight_value                     830
seller_zip_code_prefix            830
seller_city                       830
seller_state                      830
seller_lat                        830
seller_lng                        830
product_name_lenght              2528
product_desc

# Etape 9 de merge <a class="anchor" id="merge9"></a>

### merge de *df_orders_cust_orderitems_sellers_geo_products_en_order_payments* et *df_olist_order_reviews_dataset*

In [250]:
df_olist_total = df_orders_cust_orderitems_sellers_geo_products_en_order_payments.merge(df_olist_order_reviews_dataset, how='left', on='order_id')

In [251]:
df_olist_total.shape

(119151, 42)

In [252]:
df_olist_total.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 177
order_delivered_carrier_date     2086
order_delivered_customer_date    3421
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
customer_lat                        0
customer_lng                        0
order_item_id                     833
product_id                        833
seller_id                         833
shipping_limit_date               833
price                             833
freight_value                     833
seller_zip_code_prefix            833
seller_city                       833
seller_state                      833
seller_lat                        833
seller_lng                        833
product_name_lenght              2542
product_desc

In [253]:
# Sauvegarde de la df dans un fichier
df_olist_total.to_csv('df_olist_total.csv', encoding='utf-8', index=False)